In [194]:
family_sizes = [3, 4]
phase_dir = '../phased_ssc_u'
data_dir = '../split_gen_miss'
#phase_dir = '../phased_ssc'
#data_dir = '../split_gen_miss_ssc'

In [195]:
import numpy as np

all_positions = set()
family_to_individuals = dict()

for chrom in range(1, 23):
    print(chrom)

    for j in family_sizes:
        try:
            with open('%s/chr.%s.familysize.%d.families.txt' % (phase_dir, chrom, j), 'r')  as famf:
                next(famf) # skip header
                for line in famf:
                    pieces = line.strip().split('\t')
                    family_key = pieces[0]
                    family_to_individuals[family_key] = pieces[1:(1+j)]
            
            with open('%s/chr.%s.familysize.%d.phased.txt' % (phase_dir, chrom, j), 'r')  as f:
                next(f) # skip header
                for line in f:
                    pieces = line.strip().split('\t')
                    family_key = pieces[0]
                    inheritance_state = [int(x) for x in pieces[1:(2+(j*2))]]
                    start_pos, end_pos = [int(x) for x in pieces[(2+(j*2)):(4+(j*2))]]
                    all_positions.add((chrom, start_pos))
                    all_positions.add((chrom, end_pos))
        except FileNotFoundError:
            print('File not found', 'chrom', chrom, 'family size', j)
        except StopIteration:
            print('File empty', 'chrom', chrom, 'family size', j)
    
snp_positions = sorted(all_positions)
pos_to_index = dict([(x, i) for i, x in enumerate(snp_positions)])
snp_positions = np.asarray(snp_positions)
n = len(pos_to_index)
print('n', n)

family_to_index = dict([(x, i) for i, x in enumerate(family_to_individuals.keys())])
m = 4*len(family_to_index)
print('m', m)

1
File not found chrom 1 family size 4
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
n 1524947
m 2076


In [196]:
from collections import defaultdict

# load deletions
deletions = np.zeros((m, max_index), dtype=np.int8)-1

for chrom in range(1, 23):
    print(chrom)
    for j in family_sizes:
        try:
            # load deletions
            with open('%s/chr.%s.familysize.%d.phased.txt' % (phase_dir, chrom, j), 'r')  as f:
                next(f) # skip header
                for line in f:
                    pieces = line.strip().split('\t')
                    family_key = pieces[0]
                    inheritance_state = [int(x) for x in pieces[1:(2+(j*2))]]
                    start_pos, end_pos = [int(x) for x in pieces[(2+(j*2)):(4+(j*2))]]
                    start_index, end_index = pos_to_index[(chrom, start_pos)], pos_to_index[(chrom, end_pos)]
                    length = end_pos - start_pos + 1
                    
                    deletions[4*family_to_index[family_key], start_index:(end_index+1)] = inheritance_state[0]
                    deletions[(4*family_to_index[family_key])+1, start_index:(end_index+1)] = inheritance_state[1]
                    deletions[(4*family_to_index[family_key])+2, start_index:(end_index+1)] = inheritance_state[2]
                    deletions[(4*family_to_index[family_key])+3, start_index:(end_index+1)] = inheritance_state[3]

                    
        except FileNotFoundError:
            print('File not found', 'chrom', chrom, 'fammily size', j)
        except StopIteration:
            print('File empty', 'chrom', chrom, 'family size', j)


1
File not found chrom 1 fammily size 4
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [204]:
change_indices = np.where(np.any(deletions[:, 1:] != deletions[:, :-1], axis=0))[0]

plt.figure(figsize=(15, 5))
plt.imshow(deletions[:, change_indices], aspect='auto', vmin=-1, vmax=1)
plt.show()

KeyboardInterrupt: 

In [199]:
del_starts = np.where((deletions[:, :-1] != 0) & (deletions[:, 1:] == 0))
del_ends = np.where((deletions[:, :-1] == 0) & (deletions[:, 1:] != 0))
del_lengths = snp_positions[del_ends[1], 1]-snp_positions[del_starts[1]+1, 1]+1

opt_del_starts = []
opt_del_ends = []
total_bp_deleted = np.zeros((m,), dtype=int)

for ind_index, start, ind_index2, end in zip(del_starts[0], del_starts[1], del_ends[0], del_ends[1]):
    if ind_index == ind_index2:
        for i in range(start+1):
            if (deletions[ind_index, start-i] == 1) or (snp_positions[start-i, 0] != snp_positions[start-i+1, 0]):
                break
        opt_del_starts.append(start-i+1)
        
        for i in range(1, n-end-2):
            if (deletions[ind_index, end+i] == 1) or (snp_positions[end+i, 0] != snp_positions[end+i-1, 0]):
                break
        opt_del_ends.append(end+i-1)
        total_bp_deleted[ind_index] += (snp_positions[end, 1]-snp_positions[start+1, 1]+1)
    else:
        print('Error', in_index, ind_index2, start, end)
        
opt_del_lengths = snp_positions[opt_del_ends][:, 1]-snp_positions[opt_del_starts][:, 1]+1


IndexError: index 567345 is out of bounds for axis 1 with size 567345

In [ ]:
import math
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
base = 1.5
max_length = max(opt_del_lengths) 
min_length = min(del_lengths) 
print('min length', min(del_lengths) , 'max length', max(del_lengths) , 'num', len(del_lengths), 'median', np.median(del_lengths))
print('min length', min(opt_del_lengths) , 'max length', max(opt_del_lengths) , 'num', len(opt_del_lengths), 'median', np.median(opt_del_lengths))

plt.hist(del_lengths, bins=[pow(base, i) for i in range(math.ceil(math.log(max_length, base))+1)], alpha=0.5, label='conservative')
plt.hist(opt_del_lengths, bins=[pow(base, i) for i in range(math.ceil(math.log(max_length, base))+1)], alpha=0.5, label='generous')

plt.gca().set_xscale("log")
plt.legend()
plt.show()

In [200]:
plt.figure(figsize=(10, 5))
base = 1.5

start_unknown = snp_positions[del_starts[1]+1, 1]-snp_positions[opt_del_starts, 1]+1
end_unknown = snp_positions[opt_del_ends, 1]-snp_positions[del_ends[1], 1]+1

print('Percent 1', len([x for x in start_unknown if x == 1])/len(start_unknown))
print('Percent 1', len([x for x in end_unknown if x == 1])/len(end_unknown))

max_length = max(max(start_unknown), max(end_unknown))
min_length = min(min(start_unknown), min(end_unknown))
print('min length', min(start_unknown) , 'max length', max(start_unknown) , 'num', len(start_unknown), 'median', np.median(start_unknown))
print('min length', min(end_unknown) , 'max length', max(end_unknown) , 'num', len(end_unknown), 'median', np.median(end_unknown))

plt.hist(start_unknown, bins=[pow(base, i) for i in range(1, math.ceil(math.log(max_length, base))+1)], alpha=0.5, label='start uncertainty')
plt.hist(end_unknown, bins=[pow(base, i) for i in range(1, math.ceil(math.log(max_length, base))+1)], alpha=0.5, label='end uncertainty')

plt.gca().set_xscale("log")
plt.legend()
plt.show()

ValueError: operands could not be broadcast together with shapes (39581,) (288,) 

In [ ]:
import math
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams.update({'font.size': 18})

plt.figure(figsize=(8, 5))
base = 1.7
max_length = max(del_lengths) 
min_length = min(del_lengths)
med_length = np.median(del_lengths)

#print('min length', min(del_lengths) , 'max length', max(del_lengths) , 'num', len(del_lengths), 'median', med_length)
#print('min length', min(opt_del_lengths) , 'max length', max(opt_del_lengths) , 'num', len(opt_del_lengths), 'median', np.median(opt_del_lengths))

plt.hist(del_lengths, bins=[pow(base, i) for i in range(math.ceil(math.log(min_length, base)), math.ceil(math.log(max_length, base))+1)], alpha=0.5, label='conservative')

plt.gca().set_xscale("log")
plt.xlabel('Deletion Size')
plt.axvline(med_length, color='black', linestyle='--')
plt.text(med_length-15000,3200,'median=%d'%med_length,rotation=90)
plt.show()

In [ ]:
matplotlib.rcParams.update({'font.size': 18})

plt.figure(figsize=(8, 5))
base = 1.7

start_unknown = snp_positions[del_starts[1]+1, 1]-snp_positions[opt_del_starts, 1]+1
end_unknown = snp_positions[opt_del_ends, 1]-snp_positions[del_ends[1], 1]+1

#print('Percent 1', len([x for x in start_unknown if x == 1])/len(start_unknown))
#print('Percent 1', len([x for x in end_unknown if x == 1])/len(end_unknown))

max_length = max(max(start_unknown), max(end_unknown))
min_length = min(min(start_unknown), min(end_unknown))
med_length = np.median(np.concatenate((start_unknown, end_unknown)))
#print('min length', min(start_unknown) , 'max length', max(start_unknown) , 'num', len(start_unknown), 'median', med_length)
#print('min length', min(end_unknown) , 'max length', max(end_unknown) , 'num', len(end_unknown), 'median', np.median(end_unknown))

plt.hist(np.concatenate((start_unknown, end_unknown)), bins=[pow(base, i) for i in range(1, math.ceil(math.log(max_length, base))+1)], alpha=0.5)

plt.gca().set_xscale("log")
plt.xlabel('Endpoint Resolution')
plt.axvline(med_length, color='black', linestyle='--')
plt.text(med_length,0,'median=%d'%med_length,rotation=90)
plt.show()

In [ ]:
# total bases deleted
base = 1.4

max_length = max(total_bp_deleted) 
min_length = min(total_bp_deleted) 
print('min length', min_length, 'max length', max_length, 'avg', np.mean(total_bp_deleted), 'median', np.median(total_bp_deleted))
fig, ax1 = plt.subplots(figsize=(15, 5))
ax1.hist(total_bp_deleted, bins=[pow(base, i) for i in range(math.ceil(math.log(max_length, base))+1)], alpha=0.5)

ax2 = ax1.twinx()
ax2.hist(ssc_total_bp_deleted, bins=[pow(base, i) for i in range(math.ceil(math.log(max_length, base))+1)], alpha=0.5, color='red')

plt.gca().set_xscale("log")

plt.show()

In [189]:
import scipy.stats as st

#ssc_total_bp_deleted = total_bp_deleted

print(st.ttest_ind(total_bp_deleted, ssc_total_bp_deleted, axis=0, equal_var=False))

Ttest_indResult(statistic=4.126731256015412, pvalue=3.8513237528570084e-05)
